In [13]:
import torch
import transformers

print("Torch version:{}".format(torch.__version__)) # Torch version:1.12.1
print("Cuda version: {}".format(torch.version.cuda)) # Cuda version: 11.3
print("transformers version: {}".format(transformers.__version__)) # transformers 4.28.0
print("GPU 사용 가능여부: {}".format(torch.cuda.is_available()))

Torch version:1.9.1+rocm4.2
Cuda version: None
transformers version: 4.11.3
GPU 사용 가능여부: False


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy
import json 
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
tokenizer.max_model_input_sizes

{'gpt2': 1024,
 'gpt2-medium': 1024,
 'gpt2-large': 1024,
 'gpt2-xl': 1024,
 'distilgpt2': 1024}

### SFT 

In [ ]:
data_path_1_SFT = '/aiffel/aiffel/GD9/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl' 
with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

### RM

In [ ]:
data_path_2_RM = '/aiffel/aiffel/GD9/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl'
with open(data_path_2_RM, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

### PPO

In [ ]:
data_path_3_PPO = '/aiffel/aiffel/GD9/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl'
with open(data_path_3_PPO, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

In [41]:
import re

def preprocess_data(data):
    """
    데이터를 정규표현식을 사용해 전처리하는 함수.

    Parameters:
        data (list): 전처리할 데이터 리스트.

    Returns:
        list: 전처리된 데이터를 담은 리스트.
    """
    processed_data = []

    for entry in data:
        # HTML 태그 제거
        cleaned_prompt = re.sub(r'<.*?>', '', entry['prompt'])
        cleaned_completion = re.sub(r'<.*?>', '', entry['completion'])

        # ( ) 괄호와 괄호 안 내용 제거
        cleaned_prompt = re.sub(r'\(.*?\)', '', cleaned_prompt)
        cleaned_completion = re.sub(r'\(.*?\)', '', cleaned_completion)

        # 한글, 영어, 숫자, !?.'" 외의 문자 제거
        cleaned_prompt = re.sub(r'[^가-힣a-zA-Z0-9!?.]', ' ', cleaned_prompt)
        cleaned_completion = re.sub(r'[^가-힣a-zA-Z0-9!?.]', ' ', cleaned_completion)

        # !?.'" 중복 제거
        cleaned_prompt = re.sub(r'(\!|\?|\.|\'|\"){2,}', lambda x: x.group(0)[0], cleaned_prompt)
        cleaned_completion = re.sub(r'(\!|\?|\.|\'|\"){2,}', lambda x: x.group(0)[0], cleaned_completion)
        
        # "token 숫자" 패턴 제거 (숫자가 1~3자리, 주변 공백 포함)
        cleaned_prompt = re.sub(r'\btoken\s*\d{1,3}\b', '', cleaned_prompt)
        cleaned_completion = re.sub(r'\btoken\s*\d{1,3}\b', '', cleaned_completion)

        # 연속된 공백 하나로 통일
        cleaned_prompt = re.sub(r'\s+', ' ', cleaned_prompt).strip()
        cleaned_completion = re.sub(r'\s+', ' ', cleaned_completion).strip()

        # 토크나이저를 사용해 tokens 계산
        prompt_tokens = len(tokenizer.encode(cleaned_prompt, add_special_tokens=False))
        completion_tokens = len(tokenizer.encode(cleaned_completion, add_special_tokens=False))

        processed_data.append({
            'prompt': cleaned_prompt,
            'completion': cleaned_completion,
            'tokens': completion_tokens
        })

    return processed_data

In [42]:
list_data_dict = preprocess_data(list_data_dict)
list_data_dict

[{'prompt': '불고기용 고기 한우에요?',
  'completion': '저는 인공지능 챗봇이며 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우 쇠고기 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.',
  'tokens': 72},
 {'prompt': '쓰던 앱이 유료로 전환됐어',
  'completion': '어떤 앱인지 모르기 때문에 정확한 답변을 드리기 어렵습니다. 하지만 일반적으로 유료 전환된 앱은 기존 무료 앱에서 추가적인 기능이나 서비스를 제공하는 경우가 많습니다. 이 경우 유료 전환 이전 가격이 매우 저렴하거나 무료인 경우가 많으므로 앱을 계속 사용하려면 추가적인 비용을 지불해야 할 수 있습니다. 하지만 유료 전환 이후에는 기존 사용자 또는 새로운 사용자 모두가 추가 기능과 높은 품질의 서비스를 이용할 수 있게 됩니다. 따라서 앱 개발자는 유료 모델로 전환함으로써 앱의 수익을 증가시키고 더 나은 서비스를 제공할 수 있습니다.',
  'tokens': 100},
 {'prompt': '여친이랑 다툼',
  'completion': '저는 인공지능이기 때문에 여친과의 다툼에 대해서는 조언을 드릴 수는 없습니다. 하지만 다투게 되면 상대방의 말을 경청하는 태도와 서로의 감정을 존중하는 것이 중요합니다. 논리적이고 차분한 대화를 통해 문제를 해결해보세요. 그리고 서로를 이해하고 서로의 의견을 수용하는 것이 중요합니다.',
  'tokens': 54},
 {'prompt': '술 먹고 싶어',
  'completion': '술은 알코올이 함유된 음료수이며 건강에 나쁜 영향을 미칠 수 있습니다. 따라서 술을 마시기 전에 본인의 건강 상태와 함께 적당한 양을 마시는 것이 중요합니다. 술을 마시기 전에 반드시 20세 이상인

In [49]:
# 조건에 따라 데이터 필터링
filtered_data = []
for entry in list_data_dict:
    # "completion" 단어가 prompt에 포함되어 있으면 제거
    if "completion" in entry['prompt']:
        continue

    # completion의 토큰 수 계산
    completion_tokens = len(tokenizer.encode(entry['completion'], add_special_tokens=False))

    # completion의 토큰 수가 512를 초과하면 제거
    if completion_tokens > 512:
        continue

    # 조건을 만족하면 데이터 추가
    filtered_data.append(entry)

# drop
list_data_dict = filtered_data

len(list_data_dict)

47970

### SFT 진행

In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass

In [11]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

print(tokenizer)

PreTrainedTokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'})


In [12]:
from typing import Optional, Dict, Sequence

class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [13]:
@dataclass
class DataCollatorForSupervisedDataset(object): 

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [14]:
train_dataset = SFT_dataset(data_path_1_SFT='/aiffel/aiffel/GD9/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl', tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

In [56]:
print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  7182,  8139, 45211, 11488, 12817,  7643,  8711,
        25462,  9153, 15403,  9153, 27790,  8143, 32805,  9077, 14965,  9096,
        12168, 14938, 46501,   375,   378,   378,   378, 41951,   454,  9549,
        20549,   383,  8142,  7192, 14914,  7487,  8413, 39504, 18359,  8413,
         6919,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  7487,  8413, 39504, 18359,  8413,
         6919,     1])


#### Instruction 부분이 -100으로 잘 패딩이 되어있다.

In [17]:
def decode_dataset(input_ids, labels, tokenizer):

    # 디코딩 input_ids
    decoded_input = tokenizer.decode(input_ids, skip_special_tokens=True)

    # 디코딩 labels - padding_value=-100 제거
    filtered_labels = [token for token in labels if token != -100]
    decoded_labels = tokenizer.decode(filtered_labels, skip_special_tokens=True)

    return {
        "decoded_input": decoded_input,
        "decoded_labels": decoded_labels
    }

# 함수 호출
decoded_sample = decode_dataset(
    train_dataset.input_ids[0], train_dataset.labels[0], tokenizer
)

In [69]:
# Dataset 샘플 디코딩 확인
for i in range(2):  # 샘플 2개만 확인
    decoded = decode_dataset(train_dataset.input_ids[i], train_dataset.labels[i], tokenizer)
    print(f"Sample {i+1} Input:")
    print(decoded["decoded_input"])
    print(f"Sample {i+1} Labels:")
    print(decoded["decoded_labels"])

Sample 1 Input:
### Instruction(명령어):
다음 질문에 반드시 답변해라. 노 게임 노 라이프의 등장인물 소라의 성우는 누구인가?

### Response(응답):마츠오카 요시츠구
Sample 1 Labels:
마츠오카 요시츠구
Sample 2 Input:
### Instruction(명령어):
다음 질문에 반드시 답변해라. 재규어 애스턴 마틴 등의 메이커가 뉘르부르크링에 테스터 센터를 개설한 해는?

### Response(응답):해당 정보는 제공되지 않습니다. 추가 정보가 있으면 제공해 주세요.
Sample 2 Labels:
해당 정보는 제공되지 않습니다. 추가 정보가 있으면 제공해 주세요.


In [73]:
# Collator에서 반환된 데이터 샘플 확인
batch_sample = data_collator([train_dataset[i] for i in range(2)])
print("Collator Input IDs:")
print(batch_sample["input_ids"])
print("Collator Labels:")
print(batch_sample["labels"])

Collator Input IDs:
tensor([[  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
          8006, 14914,   375,  7182,  8139, 45211, 11488, 12817,  7643,  8711,
         25462,  9153, 15403,  9153, 27790,  8143, 32805,  9077, 14965,  9096,
         12168, 14938, 46501,   375,   378,   378,   378, 41951,   454,  9549,
         20549,   383,  8142,  7192, 14914,  7487,  8413, 39504, 18359,  8413,
          6919,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
          8006, 14914,   375,  7182,  8139, 45211, 11488, 12817,  7643,  8711,
         25462,  9150,  6944,  8006,  9831, 13173, 44121,  9284, 23330,  8446,
          6824, 24364,  7467, 12420,  7485,  8022, 10349, 11211, 18262,  7470,
         12138,  8704, 44113, 10604,   375,   378,   378,   378, 41951,   454,
          9549, 20549,   383,  

In [ ]:
training_args = TrainingArguments(
    output_dir="/aiffel/aiffel/GD9/KoChatGPT/test",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=5,
    prediction_loss_only=True,
    fp16 = True
    )
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

In [ ]:
trainer.train()
model.save_pretrained('/aiffel/aiffel/GD9/KoChatGPT/output_1_SFT')

In [76]:
generator = pipeline('text-generation', model='/aiffel/aiffel/GD9/KoChatGPT/output_1_SFT', tokenizer=tokenizer)

generation_args = dict(   
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n   
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)   
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

/tmp/ipykernel_106/766424849.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(train_dataset.input_ids[i]).unsqueeze(0).to(device)  # 배치 차원 추가
Input length of input_ids is 52, but ``max_length`` is set to 20.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Input length of input_ids is 70, but ``max_length`` is set to 20.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


Input: ### Instruction(명령어):
다음 질문에 반드시 답변해라. 노 게임 노 라이프의 등장인물 소라의 성우는 누구인가?

### Response(응답):마츠오카 요시츠구
Generated: 마츠오카 요시츠구 마
Expected: 마츠오카 요시츠구
Input: ### Instruction(명령어):
다음 질문에 반드시 답변해라. 재규어 애스턴 마틴 등의 메이커가 뉘르부르크링에 테스터 센터를 개설한 해는?

### Response(응답):해당 정보는 제공되지 않습니다. 추가 정보가 있으면 제공해 주세요.
Generated: 해당 정보는 제공되지 않습니다. 추가 정보가 있으면 제공해 주세요.
Expected: 해당 정보는 제공되지 않습니다. 추가 정보가 있으면 제공해 주세요.


In [21]:
# pip install bert-score
# from bert_score import score
# from bert_score import BERTScorer

In [19]:
# 캐시 삭제
torch.cuda.empty_cache()

### RM 구현

In [33]:
import os
import json
from typing import Optional
import torch
import torch.nn as nn
from torch.optim import Adam
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, AutoConfig
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
import loralib as lora

In [34]:
class GPTRM_custom(RewardModel):

    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())
        if checkpoint:
            model.gradient_checkpointing_enable()

        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained


    def save_pretrained(self, dir):
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

In [35]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

with NaiveStrategy().model_init_context():
        model = GPTRM_custom(pretrained='skt/kogpt2-base-v2', lora_rank=0, tokenizer=tokenizer).cuda()

Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [37]:
with open('/aiffel/aiffel/GD9/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)



    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 10220
after  data num: 30660
data example: 
{'prompt': '애플은 리사를 어떻게 처리했어', 'chosen': '애플이 누구인지 명확히 알 수 없어서, 리사가 누구인지와 어떤 상황에서 처리되었는지에 대한 추가적인 정보가 필요합니다. 따라서, 보다 정확한 답변을 제공할 수 없습니다.', 'rejected': '애플은 리사를 위해 고객 서비스 부서에서 고객 다양한 컴퓨터 관련 문제에 대해 응답하는 데 필요한 모든 지원을 제공했습니다. 사용자가 하드웨어 문제를 경험할 때, 전문가들은 필요한 수리(수리, 추가 부품 제공, 소프트웨어 업그레이드 등)을 제공해 드릴 수 있습니다. 또한, 사용자가 사용 방법 문제나 기타 문제를 경험할 때, 대화 상대로 사용자를 지원할 수 있는 전문 고객 서비스 직원들이 사용자에게 상담하고 도움을 주는 데 도움이 될 수 있는 정보를 제공합니다. 또한, 인터넷에서 제공되는 정보를 통해 문제를 해결하거나 고객 서비스 웹 사이트를 통해 자신의 문제를 진단할 수 있도록 하는 등 다양한 방법으로 리사를 처리해 왔습니다.'}


In [38]:
import random
random.seed(230319)
random.shuffle(total_data_ranking2chosen)
print(total_data_ranking2chosen[45])

{'prompt': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은?', 'chosen': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은 류승완의 사무실입니다.', 'rejected': '대구 영화사옥'}


In [39]:
train_data = total_data_ranking2chosen[:1000] 
eval_data = total_data_ranking2chosen[1000:1200]

print(len(train_data))
print(len(eval_data))

train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)

1000
200


100%|██████████| 200/200 [00:00<00:00, 906.75it/s]


In [40]:
idx = 1
print('#'*70)
print('## prompt ##')
print(train_data[idx]['prompt'])
print('#'*70)
print('## chosen ##')
print(train_data[idx]['chosen'])
print('#'*70)
print('## rejected ##')
print(train_data[idx]['rejected'])

######################################################################
## prompt ##
흑고래의 무게는 어느 정도야
######################################################################
## chosen ##
흑고래의 평균 몸무게는 약 25~40톤 정도이지만, 최대 몸무게는 50톤 이상에 이를 수 있습니다.
######################################################################
## rejected ##
흑고래의 무게는 매우 다양하게 달라집니다. 약 200kg에서 10톤까지 달라질 수 있습니다.


In [41]:
trainer = RewardModelTrainer(model=model,
                             strategy=NaiveStrategy(),
                             optim=Adam(model.parameters(), lr=5e-5),
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=4,
                             max_epochs=1)

In [42]:
trainer.fit(use_lora=0)

model.save_pretrained('/aiffel/aiffel/GD9/model_result/output_2_RM')

Train step of epoch 0:  38%|███▊      | 94/250 [01:19<02:14,  1.16it/s, loss=0.856]


Train step of epoch 0:  75%|███████▍  | 187/250 [02:39<00:54,  1.17it/s, loss=0.378]


Train epoch: 100%|██████████| 1/1 [03:47<00:00, 227.27s/it]


In [43]:
def inference_RM(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    print('input: %s\nreward score: %.1f'%(input_text, output_reward))

    return output_reward

input_text = '인공지능은 똥멍청이 입니다'
output_reward = inference_RM(input_text=input_text)

input: 인공지능은 똥멍청이 입니다
reward score: -0.6


In [44]:
input_text = '인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.'

output_reward = inference_RM(input_text=input_text)

input: 인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.
reward score: -0.6


In [45]:
input_text = "인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다. AI는 현대적인 컴퓨팅 혁신에서 중추적인 역할을 하며 개인과 비즈니스의 가치를 창출합니다. 예를 들어 광학 문자 인식(OCR)은 AI를 사용해 이미지 및 문서에서 텍스트 및 데이터를 추출하고, 구조화되지 않은 콘텐츠를 비즈니스에 바로 사용할 수 있게 만들고, 유용한 정보를 창출합니다."

output_reward = inference_RM(input_text=input_text)

input: 인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다. AI는 현대적인 컴퓨팅 혁신에서 중추적인 역할을 하며 개인과 비즈니스의 가치를 창출합니다. 예를 들어 광학 문자 인식(OCR)은 AI를 사용해 이미지 및 문서에서 텍스트 및 데이터를 추출하고, 구조화되지 않은 콘텐츠를 비즈니스에 바로 사용할 수 있게 만들고, 유용한 정보를 창출합니다.
reward score: -0.4


In [46]:
input_text = "인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다."

output_reward = inference_RM(input_text=input_text)

input: 인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다.
reward score: -0.3


In [47]:
torch.cuda.empty_cache()

### SFT 와 RM 모델 비교분석

In [2]:
!apt-get update
!apt-get install -y libtinfo5

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [30.9 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,296 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4,658 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [4,183 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [4,276 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [34.6 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [4,463 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,588 kB]
Get:13 http:

In [2]:
!pip install bert-score

In [4]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 KB 1.8 MB/s eta 0:00:0000:01


In [5]:
from transformers import pipeline
from bert_score import score
from evaluate import load as load_metric
from tqdm import tqdm
import os
import json

In [7]:
# 모델 경로 설정
SFT_MODEL_PATH = '/aiffel/aiffel/GD9/KoChatGPT/output_1_SFT'
RM_MODEL_PATH = '/aiffel/aiffel/GD9/model_result/output_2_RM'

def load_test_data(test_base_path, test_files):
    """테스트 데이터 로드 함수"""
    test_data = []
    for test_file in test_files:
        file_path = os.path.join(test_base_path, test_file)
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if "data" in data:
            for entry in data["data"]:
                for qas_entry in entry.get("qas", []):
                    question = qas_entry.get("question", "")
                    answer = qas_entry.get("answer", {}).get("text", "")
                    test_data.append({"prompt": question, "completion": answer})
    return test_data

def generate_completions(model_path, tokenizer, test_data, batch_size=64):
    """생성 텍스트를 반환하는 함수"""
    generator = pipeline('text-generation', model=model_path, tokenizer=tokenizer, device=0)

    PROMPT_DICT = {
        "prompt_input": (
            "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
        )
    }

    list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': entry['prompt']}) for entry in test_data]

    generated_completions = []
    for i in tqdm(range(0, len(list_prompt), batch_size), desc=f"Generating completions for {model_path}"):
        batch_prompts = list_prompt[i:i + batch_size]
        batch_results = generator(batch_prompts, **{
            "num_beams": 4,
            "repetition_penalty": 2.0,
            "no_repeat_ngram_size": 4,
            "eos_token_id": tokenizer.eos_token_id,
            "max_new_tokens": 64,
            "do_sample": True,
            "top_k": 50,
            "early_stopping": True
        })
        for res in batch_results:
            text = res[0]['generated_text']
            response = text.split("### Response(응답):")[1].strip() if "### Response(응답):" in text else text
            generated_completions.append(response)

    return generated_completions

def evaluate_metrics(generated, references):
    """BERTScore 및 ROUGE 점수 계산 함수"""
    # BERTScore 계산
    P, R, F1 = score(generated, references, lang="ko", verbose=True)

    # ROUGE 계산
    rouge = load_metric("rouge")
    rouge_results = rouge.compute(predictions=generated, references=references)

    return {
        "bert_score": {
            "precision": P.mean().item(),
            "recall": R.mean().item(),
            "f1": F1.mean().item()
        },
        "rouge": rouge_results
    }

In [9]:
from transformers import AutoTokenizer

# 토크나이저 파일 복사
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")
tokenizer.save_pretrained('/aiffel/aiffel/GD9/KoChatGPT/output_1_SFT')  # SFT 경로
tokenizer.save_pretrained('/aiffel/aiffel/GD9/model_result/output_2_RM')  # RM 경로

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('/aiffel/aiffel/GD9/model_result/output_2_RM/tokenizer_config.json',
 '/aiffel/aiffel/GD9/model_result/output_2_RM/special_tokens_map.json',
 '/aiffel/aiffel/GD9/model_result/output_2_RM/vocab.json',
 '/aiffel/aiffel/GD9/model_result/output_2_RM/merges.txt',
 '/aiffel/aiffel/GD9/model_result/output_2_RM/added_tokens.json',
 '/aiffel/aiffel/GD9/model_result/output_2_RM/tokenizer.json')

In [12]:
import torch
print(torch.__version__)  # PyTorch 버전 확인
print(torch.cuda.is_available())  # CUDA 사용 가능 여부 확인

1.9.1+rocm4.2
False


In [10]:
# 테스트 데이터 로드
TEST_BASE_PATH = "/aiffel/aiffel/GD9/dataset/test"
TEST_FILES = [f"korquad2.1_dev_{str(i).zfill(2)}.json" for i in range(5)]

test_data = load_test_data(TEST_BASE_PATH, TEST_FILES)
original_completions = [entry['completion'] for entry in test_data]

# SFT 모델 생성 및 평가
from transformers import AutoTokenizer
sft_tokenizer = AutoTokenizer.from_pretrained(SFT_MODEL_PATH)
sft_generated = generate_completions(SFT_MODEL_PATH, sft_tokenizer, test_data)
sft_metrics = evaluate_metrics(sft_generated, original_completions)

# RM 모델 생성 및 평가
rm_tokenizer = AutoTokenizer.from_pretrained(RM_MODEL_PATH)
rm_generated = generate_completions(RM_MODEL_PATH, rm_tokenizer, test_data)
rm_metrics = evaluate_metrics(rm_generated, original_completions)

# 결과 출력
print("SFT Model Metrics")
print(f"BERTScore: Precision={sft_metrics['bert_score']['precision']:.4f}, Recall={sft_metrics['bert_score']['recall']:.4f}, F1={sft_metrics['bert_score']['f1']:.4f}")
print(f"ROUGE: {sft_metrics['rouge']}")

print("\nRM Model Metrics")
print(f"BERTScore: Precision={rm_metrics['bert_score']['precision']:.4f}, Recall={rm_metrics['bert_score']['recall']:.4f}, F1={rm_metrics['bert_score']['f1']:.4f}")
print(f"ROUGE: {rm_metrics['rouge']}")

RuntimeError: No HIP GPUs are available

In [14]:
print(f"CUDA 사용 가능 여부: {torch.cuda.is_available()}")

CUDA 사용 가능 여부: False


### PPO 설계

In [48]:
from copy import deepcopy

import torch
from torch.optim import Adam
from chatgpt.models.base import RewardModel
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from transformers import AutoTokenizer

In [49]:
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained='/aiffel/aiffel/GD9/KoChatGPT/output_1_SFT', lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained='/aiffel/aiffel/GD9/model_result/output_2_RM', lora_rank=0).to(torch.cuda.current_device())

    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", 
        model_max_length=512
    )

    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [50]:
actor_optim = Adam(actor.parameters(), lr=5e-6)
critic_optim = Adam(critic.parameters(), lr=5e-6)

In [51]:
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

In [52]:
with open('/aiffel/aiffel/GD9/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

In [53]:
print(tokenize_fn('It takes something more than intelligence to act intelligently.'))

{'input_ids': tensor([[47311, 10448, 19008,  9792, 11780, 11308, 30190, 10929, 11849, 21663,
         44389,  9574, 13799,   458, 14308, 12778, 22469, 20938, 44696,   458,
         13799,   458, 14308, 12778, 11756, 18944,   389]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}


In [54]:
len(list_prompt)

12000

In [55]:
trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=1,  
                     train_batch_size=8, 
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

In [56]:
trainer.fit(list_prompt, 
            num_episodes=10,  
            max_timesteps=3,
            update_timesteps=3)

model.save_pretrained('/aiffel/aiffel/GD9/model_result/output_3_PPO')

Episode [10/10]: 100%|██████████| 3/3 [00:17<00:00,  5.69s/it]


In [57]:
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print()
    print(output)
    return output

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [
    '불고기용 고기 한우에요?', 
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?', 
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

for input_text in list_prompt:
    output = generation(input_text)


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 챗봇이므로 제가 어떤 종류의 쇠고기를 판매하는지 알 수 없습니다. 죄송합니다. srkin (쇠고기) 추가 정보가 필요합니다. 불고기용 한우는 불고기용 부위를 말합니다. geos, 또는 kinhungsin에서 각각 다른 이름입니다. geos, 또는 geos, 혹은 kinhungsin에서는 다양한 이름으로 사용됩니다. geos, 또는  Kinhungsin은 각각 다른 이름으로 사용되기도 합니다.  ninglawaid에서 각각 다른 이름으로 사용될 수 있습니다. geos, 또는  Ninglawaid에서는 주로 불고기용으로 사용되는 경우가 많습니다. geos, 또는 geos는 각각 다른 이름으로 사용될 수 있습니다. geos, 또는 geos는 각각 다른 이름으로 쓰일 수 있습니다. geos, 또는 geos는 각각 다른 이름으로 사용되기도 합니다. neos, 또는 

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):"저는 데이터에 대한 의존도를 가지고 있지 않기 때문에 제임스 닉슨의 부통령직에서 근무한 기간도에 대해 알지 못합니다. 정확한 정보와 관련해서는 "리처드 닉슨의 47대 부통령직"이라는 개인의 이름이나 연도, 인물 정보에 대한 정보가 없습니다.", 'token': 147}自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어

### Response(응답):'저는 인공지능 챗봇이므로 시카고에 있는 국제공항 정보나 위치 정보가 없습니다. 하지만 현재 시카고에는 수많은 공항이 있으므로 

### 추가 데이터 - korquad2.1

In [ ]:
base_path = "/aiffel/aiffel/GD9/dataset/train"
file_prefix = "korquad2.1_train_"
file_count = 39  # 00부터 38까지

for i in range(file_count):
    file_name = f"{file_prefix}{str(i).zfill(2)}.json"
    file_path = os.path.join(base_path, file_name)

    try:
        with open(file_path, "r", encoding="utf-8") as json_file:
            json.load(json_file)
        print(f"파일 정상 확인: {file_name}")
    except Exception as e:
        print(f"JSON 파싱 에러 발생: {file_name} - {e}")

In [ ]:
# 새로운 데이터 경로 및 파일 정보
base_path = "/aiffel/aiffel/GD9/dataset/train"
file_prefix = "korquad2.1_train_"
file_count = 39  # 00부터 38까지

# 새로운 데이터를 기존 데이터 key에 맞게 변환 후, 기존 데이터에 추가
try:
    for i in range(file_count):
        file_name = f"{file_prefix}{str(i).zfill(2)}.json"
        file_path = os.path.join(base_path, file_name)

        # 파일 열기 및 변환
        with open(file_path, "r", encoding="utf-8") as json_file:
            data = json.load(json_file)

        # 'data' 키의 내용 처리
        if "data" in data:
            for entry in data["data"]:
                for qas_entry in entry.get("qas", []):
                    question = qas_entry.get("question", "")
                    answer = qas_entry.get("answer", {}).get("text", "")

                    # prompt와 completion 생성
                    prompt = question
                    completion = answer

                    # 토크나이저를 이용해 토큰 길이 계산
                    tokens = len(tokenizer.encode(completion, add_special_tokens=False))

                    # 저장 형식에 맞게 추가
                    list_data_dict.append({
                        "prompt": prompt,
                        "completion": completion,
                        "tokens": tokens
                    })

    # 최종 결과 확인
    print(f"합쳐진 데이터 개수: {len(list_data_dict)}")
    print("첫 번째 데이터:")
    print(json.dumps(list_data_dict[:1], ensure_ascii=False, indent=2))

except Exception as e:
    print(f"새 데이터 처리 에러: {e}")

# 결과를 새 파일로 저장
output_path = "/aiffel/aiffel/GD9/KoChatGPT/data_kochatgpt/merged_data.jsonl"
try:
    with open(output_path, "w", encoding="utf-8") as output_file:
        json.dump(list_data_dict, output_file, ensure_ascii=False, indent=2)
    print(f"결과가 저장되었습니다: {output_path}")
except Exception as e:
    print(f"저장 에러: {e}")

### 새로운 데이터 셋

In [ ]:
# JSONL 파일 경로
merged_data_path = "/aiffel/aiffel/GD9/KoChatGPT/data_kochatgpt/merged_data.jsonl"

# JSONL 파일 읽어서 list_data_dict로 저장
try:
    with open(merged_data_path, "r", encoding="utf-8") as json_file:
        list_data_dict = json.load(json_file)  # JSONL 파일 읽기

    # 데이터 확인
    print(f"데이터 개수: {len(list_data_dict)}")
    print("첫 번째 데이터:")
    print(json.dumps(list_data_dict[:20], ensure_ascii=False, indent=2))
except FileNotFoundError:
    print(f"파일을 찾을 수 없습니다: {merged_data_path}")
except json.JSONDecodeError as e:
    print(f"JSON 디코딩 오류 발생: {e}")
except Exception as e:
    print(f"에러 발생: {e}")

### 전처리 후 다시 분석 

In [ ]:
# 데이터에서 tokens key 추출
tokens = [entry['tokens'] for entry in list_data_dict]

plt.figure(figsize=(8, 4))
plt.plot(tokens, marker='o') 
plt.xlabel('Index')
plt.ylabel('Tokens')
plt.title('Tokens per Index')
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
import re

def preprocess_data(data):
    """
    데이터를 정규표현식을 사용해 전처리하는 함수.

    Parameters:
        data (list): 전처리할 데이터 리스트.

    Returns:
        list: 전처리된 데이터를 담은 리스트.
    """
    processed_data = []

    for entry in data:
        # HTML 태그 제거
        cleaned_prompt = re.sub(r'<.*?>', '', entry['prompt'])
        cleaned_completion = re.sub(r'<.*?>', '', entry['completion'])

        # ( ) 괄호와 괄호 안 내용 제거
        cleaned_prompt = re.sub(r'\(.*?\)', '', cleaned_prompt)
        cleaned_completion = re.sub(r'\(.*?\)', '', cleaned_completion)

        # 한글, 영어, 숫자, !?.'" 외의 문자 제거
        cleaned_prompt = re.sub(r'[^가-힣a-zA-Z0-9!?.]', ' ', cleaned_prompt)
        cleaned_completion = re.sub(r'[^가-힣a-zA-Z0-9!?.]', ' ', cleaned_completion)

        # !?.'" 중복 제거
        cleaned_prompt = re.sub(r'(\!|\?|\.|\'|\"){2,}', lambda x: x.group(0)[0], cleaned_prompt)
        cleaned_completion = re.sub(r'(\!|\?|\.|\'|\"){2,}', lambda x: x.group(0)[0], cleaned_completion)
        
        # "token 숫자" 패턴 제거 (숫자가 1~3자리, 주변 공백 포함)
        cleaned_prompt = re.sub(r'\btoken\s*\d{1,3}\b', '', cleaned_prompt)
        cleaned_completion = re.sub(r'\btoken\s*\d{1,3}\b', '', cleaned_completion)

        # 연속된 공백 하나로 통일
        cleaned_prompt = re.sub(r'\s+', ' ', cleaned_prompt).strip()
        cleaned_completion = re.sub(r'\s+', ' ', cleaned_completion).strip()

        # 토크나이저를 사용해 tokens 계산
        prompt_tokens = len(tokenizer.encode(cleaned_prompt, add_special_tokens=False))
        completion_tokens = len(tokenizer.encode(cleaned_completion, add_special_tokens=False))

        processed_data.append({
            'prompt': cleaned_prompt,
            'completion': cleaned_completion,
            'tokens': completion_tokens
        })

    return processed_data

In [ ]:
# 조건에 따라 데이터 필터링
filtered_data = []
for entry in list_data_dict:
    # "completion" 단어가 prompt에 포함되어 있으면 제거 - 잘못 split 된 데이터들
    if "completion" in entry['prompt']:
        continue

    # completion의 토큰 수 계산
    completion_tokens = len(tokenizer.encode(entry['completion'], add_special_tokens=False))

    # completion의 토큰 수가 512를 초과하면 제거
    if completion_tokens > 512:
        continue

    # 조건을 만족하면 데이터 추가
    filtered_data.append(entry)

# drop
list_data_dict = filtered_data

len(list_data_dict)

### 다시 분석
* 노드가 끝나버려서인지 GPU 사용불가가 떠서 아쉽게도 보류.
* SFT와 RM의 결과는 당연 RM의 결과가 더 좋을 것이다. 데이터 몇 개만 뽑아 직접 확인했을 때도 그러했으니 정량평가에서도 RM의 성능이 더 좋을 것.
* 데이터 셋을 추가하면 당연 기존 데이터로 진행한 모델보다 SFT,RM,PPO 과정에서 전부 좋은 성능을 보일 것이다.